# **Differential expression analysis from publically-available RNAseq data**

## Goals
1. Introduction to publically-available RNAseq data.
2. Understanding the importance of metadata when using publically-available data and tools.
3. Data wrangling for the use of open source tools.
4. Use of Bioconductor tools through a real-world question.

## Prerequisites
- A working knowledge of R although the written code is intended to be as simple as possible for clarity.



## RNAseq
This is not intended to be a comprehensive explanation of RNAseq analysis, but by the end of this module you will have completed a simple Group A vs. Group B differential expression analysis using publically-available RNAseq data derived from [The Cancer Genome Atlas](https://cancergenome.nih.gov/), [reanalyzed to remove computational batch effects](https://www.nature.com/articles/nbt.3772), and [deposited for use](https://xenabrowser.net/datapages/?cohort=TCGA%20Pan-Cancer%20(PANCAN). This module also is a single example of how to do this analysis, but many different pipelines have been developed to give similar results. [Conesa et. al](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4728800/) published a fairly comprehensive review of RNAseq best practices and other groups have compared the effectiveness of different pipelines, such as [Soneson et al.](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-14-91) and [Teng et al.](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4842274/)

For a brief introduction to RNAseq, watch the first **8 minutes** of this video created by Josh Starmer at UNC. 

[![RNAseq intro](http://img.youtube.com/vi/tlf6wYJrwKY/0.jpg)](https://www.youtube.com/watch?v=tlf6wYJrwKY)


***
***
***



# How is RNAseq data publically-available? Isn't that a HIPPA violation?
Great question. The publication of sequence alignments would be a HIPPA violation. However, there are several ways of analyzing RNAseq data and they do not all result in raw reads or alignment files.

## *De Novo Assembly*
- Typically used when a reference genome/transcriptome is unavailable or unreliable.
- Extremely computationally intensive.
- Results contain sequence information in alignment form (.bam)
- Ex. [Trinity](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3875132/)



## Referenced-based Assembly

#### **Alignment-based**
- Can only be used when a reference genome/transcriptome is available.
- Computationally intensive, but not as much so as *de novo* assembly
- Results contain sequence information in alignment form (.bam)
- Some alignment based methods such as Tophat and HiSat2 can detect novel genes/isoforms.
- Mutation/editing events can be called from alignment file.
- Exs. [Tophat2](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2013-14-4-r36) and [HiSat2](https://ccb.jhu.edu/software/hisat2/index.shtml)
    
#### **Alignment-free**
- Requires a reference transcriptome.
- Orders of magnitude faster than alignment-based methods.
- **Resulting file does not contain sequence information**, but rather the number of reads (counts) of each transcript/gene.
- Cannot detect novel genes/isoforms.
- Exs. [Kallisto](https://www.nature.com/articles/nbt.3519) and [Salmon](https://www.nature.com/articles/nmeth.4197).







# Where can I find publically-available alignment-free RNAseq data?
Some authors choose to publish the files in the supplementary information with their papers, other deposit files into [GEO](https://www.ncbi.nlm.nih.gov/gds) or [SRA](https://www.ncbi.nlm.nih.gov/sra), and others have it hosted by browsers such as the [XENA Browser](https://xenabrowser.net/datapages/). The data that you will use in this module is counts data from [The Cancer Genome Atlas](https://cancergenome.nih.gov/) that we will download from the XENA browser. 

## TCGA
The goal of the TCGA was to understand all of the changes that lead to the formation of cancer. See the image below for details about the type and amount data generated and some of the general findings.
![TCGA](https://cancergenome.nih.gov/PublishedContent/Images/images/tcga-infographic-enlarge.__v100169753.png "TCGA")







## Encyclopedia of DNA Elements (ENCODE)

The original TCGA data was analyzed years ago using alignment-based approachs with a transcriptome file that only included ~20,000 genes. In the years since the original publication of the data, our understanding of the transcriptome has increased with large genomic studies such as [ENCODE](https://www.encodeproject.org/), which is also a repository of publically-available genomic data. ![ENCODE](https://www.encodeproject.org/static/img/classic-image.jpg "ENCODE") 

The results of ENCODE suggested that ~80% of the genome is functional and that there are closer to 60,000 genes than 20,000. Most of the other genes discovered were non-coding genes, meaning the functioning unit is the RNA rather than the RNA serving only as a meassage to make protein. That means if used the [publically-available TCGA data](https://gdac.broadinstitute.org/) than we would only be analzing 1/3 of the possible data. However, it would be unreasonable for us to reanalyze all of the TCGA data (it would cost ~$800,000 in compute time) and because it is referenced-based it would contain identifiable information, meaning we would need a data use agreement to do so. Fortunately, all of TCGA data was [reanalyzed](https://www.nature.com/articles/nbt.3772) with a newer genome annotation file that includes ~60,000 genes and ~200,000 transcripts. We will use this data that is available at from the [XENA Browser](https://xenabrowser.net/datapages/), which hosts over 1500 publically-available datasets.  

***
___
---

# The question that we will answer
## What are the differentially expressed genes between HER2 positive and Triple-Negative Breast Cancer (TNBC)?
Breast cancer is a classic example of the use of precision treatment. Patients with cancerous cells that express the estrogen receptor (ER+) receive hormone therapy and a recent [report](https://www.ncbi.nlm.nih.gov/pubmed/29860917?dopt=Abstract) demonstrates that a subset of those patients based entirely on gene expression do not benefit from chemotherapy. Likewise, patients with cancerous cells with amplified ERBB2 (HER2) are treated with a targeted therapy that has dramatically improved the prognosis of those patients. However, patients with breast cancer cells that do not express ER, HER2, or the progesteron receptor (PR) have what is known as triple-negative breast cancer (TNBC). There are currently no targeted therapies for TNBC breast cancer. One might be interested in what genes are expressed differently in HER2+ vs. TNBC given that HER2+ are typically also ER and PR negative. We will ask that question using the TCGA counts data. 



## Our pipeline
In the pipeline image, the red text indicates the names of the R packages we will use to perform the analysis, the yellow boxes indicate the data (raw or processed), and the grey indicates what we will do in each step. Note that we will use [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) to identify differentially expressed genes (DEGs). This is a highly cited package that uses a negative binomial distribution to calculate DEGs. However, there are multiple pipelines and programs to do this. [Knoweng](http://education.knoweng.org/sequenceng/) has a nice interface to explore other options, including the strengths and weaknesses of each.
![](https://github.com/davidnboone/test-image/blob/master/overview-image.jpg?raw=true "pipeline overview")

***
***
***

# Let's get started by Installing and loading the necessary R packages



In [ ]:
#you can very easily use "install.packages" or "biocLite" to install the packages and "library" to load them
#however, instead I am installing a package called pacman that will determine if a package is already installed
#if it is not it will install it and after will load it
#this method is convenient when sharing code that requires the use of others packages

if (!require("pacman")) install.packages("pacman")
pacman::p_load(R.utils, data.table, tximport, DESeq2, biomaRt, jsonlite, BiocParallel, ggplot2, gplots, RColorBrewer, devtools, pheatmap)


#it is possible that you will have problems installing and loading some packages or that you might have to respond 
#to prompts to do so correctly. If you follow the prompts you should be able to install everything. 
#If there are problems, please post on Github


# Setup a workspace
I like to create a project name as the parent directory with subdirectories to hold the downloaded raw and metadata. I then will create other subdirectories by date to host all of the analyses. This project requires several (40-50) GBs of data so I recommend linking to an external hard drive or cloud storage such as Box through Box Drive if you have a small hard drive.


In [ ]:
#after base_dir please modify the code to reflect what YOU want your base directory to be
base_dir <- "/Volumes/Oesterreich"
setwd(base_dir)

project_name <- "TCGA_Her2_TNBC_DEGs"
date <- "2019_02_06"
dir.create(file.path(project_name, "raw_data"), recursive = TRUE)
dir.create(file.path(project_name, "metadata"), recursive = TRUE)
dir.create(file.path(project_name, date), recursive = TRUE)
setwd(project_name)


In [ ]:
#Now I will do a quick check that I am in the properly created directory
getwd()

# Reproducibilty is only possible if we know the correct version of all the software and data. R has a built in function sissionInfo() that will print all of the versions of the packages and R itself. It is critical to record this metadata with your analyses. 

In [ ]:
#check that all of the packages loaded
sessionInfo()

#save that info to file
writeLines(capture.output(sessionInfo()), file.path(date, "sessionInfo.txt"))

#note normally I would use the sink command as shown below
#the sink command saves all info that is normally printed to screen
#However, sink doesn't work well in jupyter notebooks
#sink(file.path("raw_data", "analysis.package.versions.txt"))
#sessionInfo()
#sink()

# Download and decompress the data
Note we are downloading the count data and abumdance data (tpm)from the [TCGA pancan analysis](https://xenabrowser.net/datapages/?cohort=TCGA%20Pan-Cancer%20(PANCAN) on the XENA browser because this contains all of the reanalyzed TCGA data. However, for the clinical data we will download directly from the [breast cancer dataset](https://xenabrowser.net/datapages/?cohort=TCGA%20Breast%20Cancer%20(BRCA) also hosted on the XENA Browser. This is because much of the clinical data is not retained in the pancan phenotype data because every cancer type has different clinical tests that are unique. Note this will take some time. 

The final size of the files are:
- BRCA_clinicalMatrix.gz = 269 KB
- tcga_Kallisto_est_counts.gz = 3.75 GB
- tcga_Kallisto_tpm.gz = 3.95 GB

If you consistently run into download errors stating the transfer of a partial file, I suggest navigating to the appropriate xena browser webpage (https://xenabrowser.net/datapages/?cohort=TCGA%20Pan-Cancer%20(PANCAN)&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443) and download directly from there. Scroll to the bottom of the page from the link above and click on "TCGA Kallisto est_counts" or "TCGA Kallisto tpm." On those respective pages, click on the download link. You can then rename the downloaded file based on the destfile call below and move to the appropriate directories.

In [ ]:
#clinical data downloaded from 
download.file(url = "https://tcga.xenahubs.net/download/TCGA.BRCA.sampleMap/BRCA_clinicalMatrix.gz", destfile = file.path("raw_data", "tcga_brca.clinical.gz"))
download.file(url = "https://tcga.xenahubs.net/download/TCGA.BRCA.sampleMap/BRCA_clinicalMatrix.json", destfile = file.path("metadata", "tcga_brca.clinical.metadata"))


#download TCGA count data this is pancan
download.file(url = "https://toil.xenahubs.net/download/tcga_Kallisto_est_counts.gz", destfile = file.path("raw_data", "tcga_countdata.gz"))
download.file(url = "https://toil.xenahubs.net/download/tcga_Kallisto_est_counts.json", destfile = file.path("metadata", "tcga_count_metadata"))

#download TCGA TPM data this is pancan
download.file(url = "https://toil.xenahubs.net/download/tcga_Kallisto_tpm.gz", destfile = file.path("raw_data", "tcga_tpmdata.gz"))
download.file(url = "https://toil.xenahubs.net/download/tcga_Kallisto_tpm.json", destfile = file.path("metadata", "tcga_tpm_metadata"))





# Confirm that the files downloaded appropriately
When dealing with relatively large files, if at all possible you should confirm that the files downloaded completely. One possible way of doing this is through the use of [md5sum](https://en.wikipedia.org/wiki/Md5sum). This uses hashes to create a 'digital fingerprint' of the data. Sometimes these are provided by the hosts of the data. Unfortunately, the XENA browser does not supply MD5 codes, but I have made them for you. Run the code below and confirm that the hashes are identical.

- tcga_brca.clinical.gz = 'c67d5f0dea5940fd61cd270b4596a86d'
- tcga_coundata.gz = '86f8a3f1059053844428939b12b30f04'
- tcga_tpmdata.gz ='f44b67b0347e4164a8b29a6cd1a9d709'

In [ ]:
tools::md5sum("./raw_data/tcga_brca.clinical.gz")
tools::md5sum("./raw_data/tcga_countdata.gz")
tools::md5sum("./raw_data/tcga_tpmdata.gz")


In [ ]:
#the counts and TPM data are compressed files so we will use the R.utils function gunzip to decompress
#remove = FALSE so that the compressed file is not deleted in case the decompression fails
#if space is an issue have remove = TRUE
gunzip("./raw_data/tcga_countdata.gz", remove = FALSE)
gunzip("./raw_data/tcga_tpmdata.gz", remove = FALSE)
gunzip("./raw_data/tcga_brca.clinical.gz", remove = FALSE)




# Confirm the files were extracted
Run the command below to confirm that the files downloaded into the proper directories, were decompressed, and are the correct size (size is in bytes). Also, this will show you the full names of the files that you will use later in this exercise. The final file sizes of the decompressed files are:
- tcga_brca.clinical = 1.7 MB
- tcga_coundata = 14.7 GB
- tcga_tpmdata = 16.37 GB

In [ ]:
#look to see that files downloaded and extracted. Also determine the path for the appropriate clinical file
files <- list.files(recursive = TRUE)
sizes <- file.size(files)

data.frame(files = files, sizes = sizes)
           


___
___
___

# Think about another question that you would like to answer.
It can be a question based on the clinical data in the breast cancer set, or a question in another tumor type in TCGA or even normal like tissues using GTEX data. How would you start the analysis? What files do you need to download. Explore the Xena browser to determine if it contains the counts data that you need.